# Algorithms Comparison thorugh Monte Carlo Analysis
In the first script, an analysis of the effect of the parasite terms (capacitance, $C_1$, and resistance, $R_0$) is performed using the Monte Carlo technique. This way, many simulations are performed to analyze the effect of the variations or uncertainties in the parasite terms over the theoretical estimation of viscosity through the step signal response. 

In this second script, the same approach is followed but with the aim of comparing the response of different algorithms to, first, the same parasite terms, and second, to different input signals.

The content of this notebook is the following:
- Assignation of values for all the elements of the circuits and their uncertainty ranges.
- Definition of all the equations involved in the analysis.
- Definition of estimation algorithms / load of estimation models.
- Monte Carlo analysis routine for parasite terms.
- Monte Carlo analysis routine for input signals.
- Results analysis.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

from scipy import signal
from sklearn import linear_model
import pysindy as psdy

import sysidentpy as sip
from sysidentpy.model_structure_selection import FROLS as sipFROLS
from sysidentpy.basis_function._basis_function import Polynomial as sipPoly
from sysidentpy.metrics import root_relative_squared_error as sipRLSE
from sysidentpy.utils.display_results import results as sipResults

## 1. Elements of the circuit
The simulated circuit is showing here:

*Insert a diagram of the circuit*

Each dictionary holds the required values to define the associated element of the circuit. Within the dictionary there is a list called `"e_sources"`, or error sources. This list will contain all the parameters to be randomly modified in the Monte Carlo analysis. To do so, the parameter must have an associated parameter `_e`, which defined the absolute error associated to the parameter.

For example, if the diameter, `D`, of the first resistance, `R0`, must be analyzed, it must have an associated parameter `D_e`, defining the absolute uncertainty of that parameter. Also, `D` must be then included in the list `e_sources` of `R0`.

**Note**: It is important to respect the nomenclature in the dictionary of parameters. If it is a resistive element, it must start by `R`. If it is capacitive, `C`. After the identificative number, use a low bar, `_`, for the second part of the name (e.g. `R0_params`).

In [ ]:
temp_C = 25.65 # In Celsius
temp_K = temp_C + 273.15

# Input resistance
elements = {
    "R0_params": {
        "D": 500e-6,
        "L": 125e-2, # 25e-2
        "e_sources": [],
        "e": {
            "D_e": 50e-6,
            "L_e": 1e-3
        }
    },

    # First RC Branch
    "R1_params": {
        "D": 150e-6,
        "L": 10e-2,
        "e_sources": [],
        "e": {
            "D_e": 15e-6,
            "L_e": 1e-3
        }
    },
    "C1_params": {
        "D": 2e-3,
        "L": 1.0e-2,
        "E": 10e6,
        "t": 0.75e-3,
        "beta": 2.5e9,
        "e_sources": ["L"],
        "e": {
            "D_e": 0.0,
            "L_e": 0.5e-2
        }
    },

    # Second RC branch
    "R2_params": {
        "D": 150e-6,
        "L": 10e-2,
        "e_sources": [],
        "e": {
            "D_e": 15e-6,
            "L_e": 1e-3
        }
    },
    "C2_params": {
        "D": 2e-3,
        "L": 10e-2,
        "E": 10e6,
        "t": 0.75e-3,
        "beta": 2.5e9,
        "e_sources": [],
        "e": {
            "D_e": 5e-4,
            "L_e": 1e-3
        }
    },
}

In [ ]:
# Use the above dictionaries to define the ranges
ranges_dict = {}
for ky, element in elements.items():
    ranges_dict[ky] = {}
    for param in set(element.keys())-set(["e_sources", "e"]):
        if param in element["e_sources"]:
            ranges_dict[ky][param] = (element[param] - element["e"][param+"_e"],
                                      element[param] + element["e"][param+"_e"])
        else: 
            ranges_dict[ky][param] = (element[param], element[param])

print(ranges_dict)

## 2. Equations

### Input Signals

In [ ]:
def constant_signal(t, A=1.0, k=1.0):
    return A

def sinusoidal_signal(t, A=1.0, k=1.0):
    return A*np.sin(k*t)

def triangular_signal(t, A=1.0, k=0.5):
    return A*(signal.sawtooth(2*np.pi*k*t, 0.5)+1.0) / 2

### Physical Equations

In [ ]:
class calc_water_viscosity:
    """
    Calculates the viscosity at some temperature, in Kelvin.

    This is a functor, so it must first be declared (changing the parameters if
    needed), and, after that, it can be called as any other function.
    """
    def __init__(self, A=2.414e-5, B=247.8, C=140):
        self.A = A
        self.B = B
        self.C = C
    def __call__(self, temp_K):
        return self.A * 10 ** (self.B / (temp_K - self.C))

def calc_cil_resistance(D, L, visc):
    """
    Calculates the hydraulic resistance for a cylindrical tube,
    for a given viscosity.
    """
    return 8*visc*L / (np.pi*((D/2)**4))

def calc_cil_capacitance(D, L, beta, E, t):
    """
    Calculates the theoretical capacitance of a cylindrical tube.
    """
    # Beta := Bulk Modulus of Water
    V0 = np.pi*(D/2)**2 * L
    return (V0 / beta)*( 1 + ((beta*D)/(E*t)) )

def circuit_dyn_eqs(t, dP, args): # R0, R1, R2, C1, C2, Pin
    """
    Calcualtes the differential values of pressure drop over the
    two RC branches of the circuit.
    """
    P1, P2 = dP # Integration of previous dP1, dP2
    R0 = args["R0"]
    R1 = args["R1"]
    R2 = args["R2"]
    C1 = args["C1"]
    C2 = args["C2"] 
    # Pin = args["Pin"]
    Pin = args["Pin"](t)

    dP1 = (Pin - P1)/(R0 * C1) - P1 / (R1 * C1)
    dP2 = (P1 - P2)/(R2 * C2)
    return [dP1, dP2]


## 3. Estimation Algorithms
There are six compared viscosity estimation algorithms: two based on classical techniques (N4SID and SINDy), and four based on deep learning techniques. Within the deep learning models, one is based on Convolutional Neural Networks (CNN), and the other is based on Long-Short Term Memory (LSTM) networks. The two remaining deep learning models are the quantized versions of the CNN and LSTM based models. This is done in this way such that classical techniques, which are easily implemented as embedded software, can be benchmarked under the conditions that the deep learning techniques would have as embedded software. 

- One observed limitation for SINDy is its dependence on the data points. If it is small, the quality of the discovered models (and the coefficients) can be very poor when compared with other techniques.

In [ ]:
def estimate_visc(t, P2_t, R2, C2, visc):
    """
    Estimates the viscosity from an step response of the target 
    microfluidic RC circuit. It assumes that there is not noise.

    The viscosity is asked as argument because R2 is assumed to 
    contain the viscosity. 
    """
    P_ss = P2_t[-1]
    if np.abs(P2_t[-1]-P2_t[-2]) > np.abs(0.02*P2_t[-1]):
        print("W: steady state value was not steady calculating visc.")
    P_tau = 0.632 * P_ss
    # Get closest time, t
    tau = t[ np.argmin(np.abs(np.subtract(P2_t, P_tau))) ]

    Rg = R2 / visc
    return tau / (Rg*C2)

def estimate_visc_lasso(t, P1_t, P2_t, alpha=0.1):
    # Format inputs
    XY = np.transpose(np.vstack([P1_t, P2_t]))
    # Identify
    reg = linear_model.Lasso(alpha=alpha)
    reg.fit(XY)
    return reg.coef_, reg.intercept_

def estimate_visc_sindy(t, P1_t, P2_t):
    # Identify
    model = psdy.SINDy()
    model.fit(P2_t, u=P1_t, t=t[1]-t[0])
    # Get tau as the mean estimated from both variables
    # Model: dP2/dt = tau_a*P1 - tau_b*P2 -> tau = (tau_a+tau_b)/2
    return 1/np.mean(np.abs(model.coefficients()[0][1:3]))

def estimate_visc_n4sid(t, P1_t, P2_t):
    pass

def estimate_visc_lms(t, P1_t, P2_t):
    x = np.reshape(P1_t, [np.shape(P1_t)[0], 1])
    y = np.reshape(P2_t, [np.shape(P2_t)[0], 1])
    basis_function = sipPoly(degree=1)

    model = sipFROLS(
        order_selection=False,
        n_terms=3,
        extended_least_squares=False,
        ylag=[1],
        xlag=[1],
        estimator="least_mean_squares",
        basis_function=basis_function,
    )
    model.fit(X=x, y=y)

    r = pd.DataFrame(
        sipResults(
            model.final_model,
            model.theta,
            model.err,
            model.n_terms,
            err_precision=8,
            dtype="sci",
        ),
        columns=["Regressors", "Parameters", "ERR"],
    )
    # print(r)
    # print(model.theta)
    incT = t[1]-t[0]
    tau1 = incT / model.theta[0] - incT
    tau2 = model.theta[1] # (model.theta[1]*incT) / (1-model.theta[1])
    # print(tau1, tau2)
    return np.mean(np.abs([model.theta[0], model.theta[2]])) # np.mean(np.abs([tau1, tau2]))

In [ ]:
# Encapsulate all algorithms inside a dictionary
alg_dict = {
    "sindy": estimate_visc_sindy,
    "lms": estimate_visc_lms
}

## 4. Monte Carlo Analysis - Parasite Terms
For each of the element of the circuit, there are some defined ranges of values that the element can take. The objective of this analysis is to study how much will affect each of the variations to the output estimation of viscosity.

The analysis is performed by running a large number of simulations (proportional to the complexity of the system and the number of uncertain variables), propagating each variation to an objective value. For example, if the uncertainty source is one dimension of a resistance, the resistance will be calculated for each simulation with a dimension randomly chosen within the defined range. For each simulation, the viscosity will be calculated. The variation at viscosity is then compared with the variation of the input parameter, producing a sensibility estimation between both of them.

In this case, the objective parameter is the parasite capacitance, although the rest of the parameters can be analyzed with this same setup.

In [ ]:
Pin = 20e4 # Pressure input in Pa: 20e4 Pa = 2000 mbar

t_sim = 10 # Must be large enough to reach steady state
t_points = 1000 # This affects to the accuracy estimating tau
N = 100 # Number of simulations


visc_calc = calc_water_viscosity()
visc = visc_calc(temp_K)

results = {}
for ky in ranges_dict.keys():
    results[ky.split("_")[0]] = []
for alg_name in alg_dict.keys():
    results[alg_name+"_est_visc"] = []
    results[alg_name+"_tau2"] = []

for n in tqdm(range(N)): # For progress bar
    # Get the values of the parameters
    # sim_params = { "Pin": Pin }
    sim_params = { "Pin": sinusoidal_signal }

    for ky, element_ranges in ranges_dict.items():
        # Extract the random values for the parameters
        args = {}
        for param, param_ranges in element_ranges.items():
            args[param] = np.max([0.0, np.random.uniform(*param_ranges)])
        # Resistance
        if ky[0] == "R":
            sim_params[ky.split("_")[0]] = calc_cil_resistance(**args, visc=visc)
        # Capacitance
        elif ky[0] == "C":
            sim_params[ky.split("_")[0]] = calc_cil_capacitance(**args)
        else: 
            print("Error: element {} not recognized".format(ky))
            exit()
        
    # Perform the simulation
    sol = solve_ivp(circuit_dyn_eqs, [0, t_sim], [0, 0], args=(sim_params,), 
                    dense_output=True)
    
    # Save the results
    t = np.linspace(0, t_sim, t_points)
    P1_t = sol.sol(t)[0]
    P2_t = sol.sol(t)[1]

    # If you want to print some curves, don't make N very large
    # plt.plot(t, P2_t)
    # plt.plot(t, P1_t)
    # plt.show()

    for alg_name in alg_dict.keys():
        # Estimate viscosity
        R2g = sim_params["R2"] / visc
        tau2 = alg_dict[alg_name](t, P1_t, P2_t)
        est_visc = tau2 / (R2g*sim_params["C2"])
        # Save the results
        results[alg_name+"_tau2"].append(tau2)
        results[alg_name+"_est_visc"].append(est_visc)

    # Update the results dict
    for ky in ranges_dict.keys():
        results[ky.split("_")[0]].append( sim_params[ky.split("_")[0]] )

res_df = pd.DataFrame.from_dict(results)

## 4. Results Analysis
The calculated stats are the following:
- Mean $C_1$ to mean $C_2$ ratio: Percentage that mean $C_1$ suposses to mean $C_2$. This is, how big is (average) parasite capacitance, $C_1$, compared to the main measurement capacitance, $C_2$.
- $C_1$ approx. perc. deviation: Percentage that indicates how much $C_1$ varies from its mean value along the Monte Carlo analysis.
- Est. viscosity approx. perc. deviation: How much the estimated viscosity varies from its mean value when varying the input values. This indicates how sensible it is to the change in the, in this case, parasite capacitance.
- Est. viscosity MAE: Mean Absolute Error of the viscosity estimation with respect to the real value of viscosity.

### How to interpret the values?
If a high variation of the parasite capacitance, $C_1$, produces a small variation of the estimated viscosity deviation, it means that the sensibility of the calculation is low. This is, the parasite capacitance doesn't affect much to its **precision**. To complete the analysis, it must be observed if it affects to the **accuracy** of the estimation. For example, it could induce an offset that is constant (precision wouldn't be affected), but makes the mean value of the estimate deviate from the real value. This is done by calculating the MAE, which should also remain low for "big" parasite capacitances.

It can be observed that the effect of the parasite capacitance, $C_1$, alone is low. This is because the problems comes with the time response of the parasite capacitance, and not only the capacitance itself. This means that $R_0$ is also important in this analysis. If this resistance is high, $C_1$ takes more time to fill up, affecting negatively to the estimate of viscosity.

In [177]:
print("Real viscosity: {:.5e}".format(visc))

# print(res_df)

print("Mean values:")
print(res_df.mean())

print("\nStandard Deviation values:")
print(res_df.std())

# Inputs variations
print("\nMean C1 to mean C2 ratio: {:.3f} %".format(
    (1-np.abs((res_df["C1"].mean() - res_df["C2"].mean()) \
           / res_df["C2"].mean())) * 100
))
print("C1 approx. perc. deviation: {:.3f} %".format(
    res_df["C1"].std() / res_df["C1"].mean() * 100
))

for alg_name in alg_dict.keys():
    print("\nAlgorithm: {}".format(alg_name))
    # Standard deviation for est. visc. in percentage
    print("[Precision] Est. viscosity approx. perc. deviation: {:.3f} %".format(
        res_df[alg_name+"_est_visc"].std() / res_df[alg_name+"_est_visc"].mean() * 100
    ))
    # Mean estimation difference to real viscosity
    print("[Accuracy]  Est. viscosity MAE: {:.3f} %".format(
        np.abs((visc - res_df[alg_name+"_est_visc"].mean()) / visc) * 100
    ))

Real viscosity: 8.77386e-04
Mean values:
R0                7.149584e+11
R1                7.061317e+12
C1                8.436317e-15
R2                7.061317e+12
C2                8.390147e-14
sindy_est_visc    1.340122e-03
sindy_tau2        9.049174e-01
lms_est_visc      4.212909e-04
lms_tau2          2.844767e-01
dtype: float64

Standard Deviation values:
R0                1.226853e-04
R1                0.000000e+00
C1                2.458348e-15
R2                0.000000e+00
C2                1.268536e-29
sindy_est_visc    8.663100e-04
sindy_tau2        5.849759e-01
lms_est_visc      1.626696e-08
lms_tau2          1.098427e-05
dtype: float64

Mean C1 to mean C2 ratio: 10.055 %
C1 approx. perc. deviation: 29.140 %

Algorithm: sindy
[Precision] Est. viscosity approx. perc. deviation: 64.644 %
[Accuracy]  Est. viscosity MAE: 52.740 %

Algorithm: lms
[Precision] Est. viscosity approx. perc. deviation: 0.004 %
[Accuracy]  Est. viscosity MAE: 51.983 %


### Results

1. How well the algorithms estimate the viscosity?
2. What is their robustness to parasite terms?
3. What is their robustness against noise?